# Multiplication Matrix method


In this document, we will exploit the property:
\begin{align*}
C_x 
   \begin{bmatrix}
      1 & x & \dots & x^{k-1} \\
      \vdots & \vdots & \vdots \\
      x^{k-1} & x^{k} & \dots & x^{2k-2}
   \end{bmatrix}
   &= 
   \begin{bmatrix}
      x & x^2 & \dots & x^{k} \\
      \vdots & \vdots & \vdots \\
      x^{k} & x^{k+1} & \dots & x^{2k}
   \end{bmatrix}.
\end{align*}

Let $M$ and $M_+$ be the two matrices so that we can write concisely, $C_x M = M_+$. Now, consider $M' = \sum_{i=1}^k \pi_i M(x_i)$, where $x_i$ are the solutions of $x$: The entries of $M'$ and $M'_+$ are observable. Furthermore, we know through algebraic geometry that $C_x$ is uniquely determined and the eigenvalues of $C_x$ are the values of $x_i$!

In [40]:
import numpy as np
from numpy.linalg import inv, pinv, eigvals
np.set_printoptions(precision=3, suppress=True)
from IPython.display import display

Let's try to derive a multiplication matrix given a set of solutions.

In [23]:
def multiplication_matrix(d, xs, pis = None):
    """
    Construct a d x d multiplication matrix as shown above, 
    using the values xs = (x_1, ..., x_k) and pis = (pi_1, ... pi_k).
    """
    # Initialize pis appropriately
    if pis is not None:
        assert len(pis) == len(xs)
    else:
        pis = np.ones(len(xs))/len(xs)
    
    M = np.zeros((d,d))
    for i in xrange(d):
        for j in xrange(d):
            for pi, x in zip(pis, xs):
                M[i,j] += pi * (x**(i+j))
    return M

def multiplication_matrix_plus(d, xs, pis = None):
    """
    Construct a d x d multiplication matrix as shown above, 
    using the values xs = (x_1, ..., x_k) and pis = (pi_1, ... pi_k).
    """
    # Initialize pis appropriately
    if pis is not None:
        assert len(pis) == len(xs)
    else:
        pis = np.ones(len(xs))/len(xs)
    
    M = np.zeros((d,d))
    for i in xrange(d):
        for j in xrange(d):
            for pi, x in zip(pis, xs):
                M[i,j] += pi * (x**(i+j+1))
    return M

Now, let us try to solve a simple case.

In [48]:
d, xs, pis = 3, [1, 2, 3], None
M = multiplication_matrix(d, xs, pis)
M_ = multiplication_matrix_plus(d, xs, pis)
display(M)
display(M_)
C = pinv(M).dot(M_)
display(C)
xs_ = eigvals(C)
display(xs_)

array([[  1.   ,   2.   ,   4.667],
       [  2.   ,   4.667,  12.   ],
       [  4.667,  12.   ,  32.667]])

array([[  2.   ,   4.667,  12.   ],
       [  4.667,  12.   ,  32.667],
       [ 12.   ,  32.667,  92.   ]])

array([[ -0.,  -0.,   6.],
       [  1.,   0., -11.],
       [ -0.,   1.,   6.]])

array([ 1.,  2.,  3.])

What happens if we increase the number of moments we have?

In [72]:
d, xs, pis = 5, [-0.5, .25, .9], None
M = multiplication_matrix(d, xs, pis)
M_ = multiplication_matrix_plus(d, xs, pis)
display(M)
display(M_)
C = pinv(M).dot(M_)
display(C)
xs_ = eigvals(C)
display(xs_)

array([[ 1.   ,  0.217,  0.374,  0.207,  0.241],
       [ 0.217,  0.374,  0.207,  0.241,  0.187],
       [ 0.374,  0.207,  0.241,  0.187,  0.182],
       [ 0.207,  0.241,  0.187,  0.182,  0.157],
       [ 0.241,  0.187,  0.182,  0.157,  0.145]])

array([[ 0.217,  0.374,  0.207,  0.241,  0.187],
       [ 0.374,  0.207,  0.241,  0.187,  0.182],
       [ 0.207,  0.241,  0.187,  0.182,  0.157],
       [ 0.241,  0.187,  0.182,  0.157,  0.145],
       [ 0.187,  0.182,  0.157,  0.145,  0.128]])

array([[ 0.025,  0.06 , -0.064, -0.023, -0.044],
       [ 0.921, -0.143,  0.227, -0.006,  0.091],
       [-0.143,  0.521,  0.282,  0.382,  0.288],
       [ 0.227,  0.282,  0.27 ,  0.249,  0.224],
       [-0.006,  0.382,  0.249,  0.296,  0.237]])

array([ 0.9 , -0.5 ,  0.25,  0.  ,  0.  ])

So, we find that with extra moments, the pseudo-inverse gives us solutions that are extra zeros (comes from a rank deficiency).

Another way to see this is that the multiplication matrix of the rank-deficient problem, $C'$, has the structure:
\begin{align*}
C' &= \Omega^{-1} \begin{bmatrix} C & 0 \\ 0 & 0 \end{bmatrix} \Omega,
\end{align*}
where $\Omega$ is any invertible matrix. Thus, we have the following relation:
\begin{align*}
\Omega^{-1} \begin{bmatrix} C & 0 \\ 0 & 0 \end{bmatrix} \Omega M &= M_+ \\
\begin{bmatrix} C & 0 \\ 0 & 0 \end{bmatrix} \Omega M &= \Omega M_+ \\
C \Omega_k M &= \Omega_k M_+,
\end{align*}
where $\Omega_k$ is the $k \times d$ sub-matrix of $\Omega$.

In other words, there is some projection $\Omega_k$ that reduces the system to solving two $k \times d$ matrices.

A random unitary transformation should recover it for us though.

In [86]:
import scipy
import scipy.linalg
from util import orthogonal
k = 3
U = orthogonal(d)[:k,:]
N = U.dot(M).dot(U.T)
N_ = U.dot(M_).dot(U.T)
display(N)
display(N_)
D = inv(N).dot(N_)
display(D)
ys_ = eigvals(D)
display(ys_)

array([[ 0.023, -0.099,  0.035],
       [-0.099,  0.448, -0.075],
       [ 0.035, -0.075,  0.236]])

array([[ 0.018, -0.081,  0.017],
       [-0.081,  0.351, -0.111],
       [ 0.017, -0.111, -0.066]])

array([[ -8.156,  29.003, -29.342],
       [ -1.868,   6.751,  -6.387],
       [  0.691,  -2.639,   2.056]])

array([ 0.9 ,  0.25, -0.5 ])